# 单电机实验 demo

## 初始化电机

In [1]:
from robodyno.components import Motor
from robodyno.interfaces import CanBus
can = CanBus()

motor = Motor(can, 0x10)

## 使能

In [2]:
motor.enable()

## 控制

In [3]:
import ipywidgets as widgets
from IPython.display import display
import bqplot.pyplot as plt
import time 
import threading
import numpy as np

init_time = time.time()

thread_running = True

cmode, imode = motor.get_mode()[0].value
origin_pos = motor.get_pos()

target_selector = widgets.ToggleButtons(
    options=[('位置', 3), ('速度', 2), ('力矩', 1)],
    description='控制对象:',
    value = cmode
)
mode_options = [
    [], [('直接', 11)],
    [('直接', 21), ('匀加减速', 22)],
    [('直接', 31), ('滤波', 33), ('匀速', 35)],
]
mode_selector = widgets.ToggleButtons(
    options=mode_options[target_selector.value],
    description='控制模式:',
    value = cmode*10+imode
)

labels = ['', '力矩(Nm)', '速度(rad/s)', '位置(rad)']
slider_label = widgets.Label(value=labels[target_selector.value])
pos_slider = widgets.FloatSlider(
    min=-3.14,
    max=3.14,
    step=0.01,
    readout_format='.2f',
)
vel_slider = widgets.FloatSlider(
    min=-3.14,
    max=3.14,
    step=0.01,
    readout_format='.2f',
)
torque_slider = widgets.FloatSlider(
    min=-0.3,
    max=0.3,
    step=0.001,
    readout_format='.3f',
)
sliders = [None, torque_slider, vel_slider, pos_slider]
slider_box = widgets.HBox(
    [slider_label, sliders[target_selector.value]],
    layout={'margin': '24px 0 12px 0'}
)

vbus_label = widgets.Label(value='电压: {:.2f} V'.format(0))
temp_label = widgets.Label(value='温度: {:.2f} °C'.format(0))
pos_label = widgets.Label(value='位置: {:.4f} rad'.format(0))
vel_label = widgets.Label(value='速度: {:.4f} rad/s'.format(0))
torque_label = widgets.Label(value='力矩: {:.4f} Nm'.format(0))
info_label_box = widgets.VBox([
    widgets.HBox([vbus_label, temp_label]), 
    widgets.HBox([pos_label, vel_label, torque_label])
])

plot_selector = widgets.ToggleButtons(
    options=[('位置', 3), ('速度', 2), ('力矩', 1)],
    description='实时曲线:',
)

plot_x_vals = []
plot_y_vals = [[], [], [], []]

fig = plt.figure(animation_duration=0)
axes_options = {"x": {"label": "时间(s)"}, "y": {"label": "位置(rad)"}}
lines = plt.plot(x=plot_x_vals, y=plot_y_vals[plot_selector.value], axes_options=axes_options)

def on_target_change(change):
    global origin_pos
    if change.new == 3:
        origin_pos = motor.get_pos()
        motor.set_pos(origin_pos)
    elif change.new == 2:
        motor.set_vel(0)
    elif change.new == 1:
        motor.set_torque(0)
    mode_selector.options = mode_options[change.new]
    slider_label.value = labels[change.new]
    sliders[change.new].value = 0
    slider_box.children = [slider_label, sliders[change.new]]
target_selector.observe(on_target_change, names='value')

def on_mode_change(change):
    if change.new == 31:
        pos_slider.continuous_update = False
        motor.position_mode()
    elif change.new == 33:
        pos_slider.continuous_update = True
        motor.position_filter_mode(3)
    elif change.new == 35:
        pos_slider.continuous_update = False
        motor.position_track_mode(2,0.5,0.5)
    elif change.new == 21:
        motor.velocity_mode()
    elif change.new == 22:
        motor.velocity_ramp_mode(0.5)
    elif change.new == 11:
        motor.torque_mode()
mode_selector.observe(on_mode_change, names='value')

def on_pos_slider_change(change):
    motor.set_pos(change.new + origin_pos)
pos_slider.observe(on_pos_slider_change, names='value')
def on_vel_slider_change(change):
    motor.set_vel(change.new)
vel_slider.observe(on_vel_slider_change, names='value')
def on_torque_slider_change(change):
    motor.set_torque(change.new)
torque_slider.observe(on_torque_slider_change, names='value')

display(target_selector, mode_selector, slider_box, info_label_box, plot_selector, fig)

def read_pos_thread():
    while thread_running:
        vbus = motor.get_voltage()
        temp = motor.get_temperature()
        pos, vel, torque = motor.get_feedback()
        vbus_label.value='电压: {:.2f} V'.format(vbus)
        temp_label.value='温度: {:.2f} °C'.format(temp)
        pos_label.value='位置: {:.4f} rad'.format(pos)
        vel_label.value='速度: {:.4f} rad/s'.format(vel)
        torque_label.value='力矩: {:.4f} Nm'.format(torque)
        
        plot_y_vals[1].append(torque)
        plot_y_vals[2].append(vel)
        plot_y_vals[3].append(pos)
        plot_x_vals.append((time.time() - init_time))
        
        lines.x = plot_x_vals
        lines.y = plot_y_vals[plot_selector.value]
        plt.ylabel(labels[plot_selector.value])

        time.sleep(0.05)

fig_thread = threading.Thread(target=read_pos_thread)

fig_thread.start()

ToggleButtons(description='控制对象:', options=(('位置', 3), ('速度', 2), ('力矩', 1)), value=3)

ToggleButtons(description='控制模式:', index=2, options=(('直接', 31), ('滤波', 33), ('匀速', 35)), value=35)

ToggleButtons(description='实时曲线:', options=(('位置', 3), ('速度', 2), ('力矩', 1)), value=3)

Figure(axes=[Axis(label='时间(s)', scale=LinearScale()), Axis(label='位置(rad)', orientation='vertical', scale=Lin…

## 停止

In [4]:
thread_running = False
fig_thread.join()

## 电机失能

In [5]:
motor.disable()

## 断开CAN总线

In [6]:
can.disconnect()